### Grab the Data
1. Get exoplanet data from MAST
2. Filter to systems with more than 3 planets
3. Determine planet order within each system using orbital period and add as a new column
4. Classify planets into categories and add as a new column

In [1]:
#from astroquery.nasa_exoplanet_archive import NasaExoplanetArchive
from astroquery.ipac.nexsci.nasa_exoplanet_archive import NasaExoplanetArchive
import pandas as pd

# Query confirmed planets discovered
results = NasaExoplanetArchive.query_criteria(
    table="pscomppars",
    select="pl_name,hostname, st_spectype, pl_orbper,pl_rade,disc_facility"
)

# Convert to pandas DataFrame
df = results.to_pandas()

# Count planets per system using correct 'hostname' field
host_counts = df['hostname'].value_counts()
multi_hosts = host_counts[host_counts > 3].index
multi_df = df[df['hostname'].isin(multi_hosts)]

# Sort and assign planet order
multi_df = multi_df.sort_values(['hostname', 'pl_orbper'])
multi_df['planet_order'] = multi_df.groupby('hostname').cumcount() + 1

#Make a new column classifying each planet
def classify_planetType(planetERadii):
    if pd.isna(planetERadii):
        return 'Unknown'
    elif planetERadii > 13:
        return 'Super Jupiter'
    elif planetERadii > 6:
        return 'Gas Giant'
    elif planetERadii > 4:
        return 'Neptune-Like'
    elif planetERadii > 2:
        return 'Mini-Neptune'
    elif planetERadii > 1.25:
        return 'Super-Earth'
    elif planetERadii > 0.8:
        return 'Terran'
    else:
        return 'Subterran'

multi_df['pl_type'] = multi_df['pl_rade'].apply(classify_planetType)

# Reformat columns
final_df = multi_df[['hostname', 'st_spectype', 'pl_name', 'planet_order', 'pl_orbper', 'pl_rade', 'pl_type', 'disc_facility']]

# Show results
print(f"Found {len(final_df)} planets in {final_df['hostname'].nunique()} multi-planet systems (more than 3 planets)")
print(final_df.head())


Found 541 planets in 121 multi-planet systems (more than 3 planets)
     hostname st_spectype   pl_name  planet_order    pl_orbper  pl_rade  \
4925   55 Cnc         G8V  55 Cnc e             1     0.736547    1.875   
3720   55 Cnc         G8V  55 Cnc b             2    14.651600   13.900   
512    55 Cnc         G8V  55 Cnc c             3    44.398900    8.510   
3726   55 Cnc         G8V  55 Cnc f             4   259.880000    7.590   
4923   55 Cnc         G8V  55 Cnc d             5  5574.200000   13.000   

            pl_type           disc_facility  
4925    Super-Earth    McDonald Observatory  
3720  Super Jupiter        Lick Observatory  
512       Gas Giant    McDonald Observatory  
3726      Gas Giant  Multiple Observatories  
4923      Gas Giant        Lick Observatory  


### Determine planet sequences
1. Sort the data by planet order
2. Create a text sequence of planet types in each system
3. Make a new data frame of the star and its sequence of planet types

In [2]:
df_sorted = final_df.sort_values(['hostname', 'planet_order'])
# Combine the sequence of planet types for each system into one string
pattern_df = df_sorted.groupby('hostname')['pl_type'].apply(lambda x: '>'.join(x)).reset_index()
pattern_df.rename(columns={'pl_type': 'planet_type_sequence'}, inplace=True)


In [3]:
print(pattern_df.head()) 

         hostname                               planet_type_sequence
0          55 Cnc  Super-Earth>Super Jupiter>Gas Giant>Gas Giant>...
1  Barnard's star            Subterran>Subterran>Subterran>Subterran
2          DMPP-1  Super-Earth>Super-Earth>Mini-Neptune>Neptune-Like
3         GJ 3293  Super-Earth>Neptune-Like>Mini-Neptune>Neptune-...
4        GJ 667 C  Mini-Neptune>Super-Earth>Super-Earth>Super-Ear...


### Count How Many Systems Have Each Planet Type Sequence
1. Count number of systems with this planet type sequence value
2. Make a new data frame with the sequence and the count

In [4]:
pattern_counts = pattern_df['planet_type_sequence'].value_counts().reset_index()
pattern_counts.columns = ['planet_type_sequence', 'num_systems']

print(pattern_counts.head())

                                planet_type_sequence  num_systems
0  Super-Earth>Mini-Neptune>Mini-Neptune>Mini-Nep...           13
1  Super-Earth>Mini-Neptune>Mini-Neptune>Mini-Nep...            4
2         Terran>Super-Earth>Super-Earth>Super-Earth            4
3  Super-Earth>Mini-Neptune>Mini-Neptune>Super-Earth            3
4  Mini-Neptune>Mini-Neptune>Mini-Neptune>Mini-Ne...            3


### Determine "common" planet type sequences
1. Filter out planet sequences that only happen once
2. See how many systems (or hosts) have a sequence that exists elsewhere

In [5]:
# Only show planet type sequences shared by more than one system
common_patterns = pattern_counts[pattern_counts['num_systems'] > 1]
print(common_patterns)


                                 planet_type_sequence  num_systems
0   Super-Earth>Mini-Neptune>Mini-Neptune>Mini-Nep...           13
1   Super-Earth>Mini-Neptune>Mini-Neptune>Mini-Nep...            4
2          Terran>Super-Earth>Super-Earth>Super-Earth            4
3   Super-Earth>Mini-Neptune>Mini-Neptune>Super-Earth            3
4   Mini-Neptune>Mini-Neptune>Mini-Neptune>Mini-Ne...            3
5       Terran>Mini-Neptune>Mini-Neptune>Mini-Neptune            3
6               Terran>Terran>Super-Earth>Super-Earth            2
7         Terran>Super-Earth>Mini-Neptune>Super-Earth            2
8             Terran>Mini-Neptune>Terran>Mini-Neptune            2
9   Super-Earth>Super-Earth>Mini-Neptune>Mini-Nept...            2
10  Super-Earth>Super-Earth>Super-Earth>Mini-Neptu...            2
11   Super-Earth>Super-Earth>Mini-Neptune>Super-Earth            2
12         Super-Earth>Super-Earth>Terran>Super-Earth            2
13  Super-Earth>Mini-Neptune>Neptune-Like>Neptune-...         

In [6]:
total = common_patterns['num_systems'].sum()
print(total)

50


### See which host stars have these "common" planet sype sequences
1. Join dataaframes so that you can see the name of the host stars and their sequences together.

In [7]:
df_with_common_patterns = pattern_df[pattern_df['planet_type_sequence'].isin(common_patterns['planet_type_sequence'])]
print(df_with_common_patterns[:50])
print(df_with_common_patterns.count())

#print(set(pattern_df['planet_type_sequence']) & set(common_patterns['planet_type_sequence']))


        hostname                               planet_type_sequence
8      HD 108236  Super-Earth>Mini-Neptune>Mini-Neptune>Mini-Nep...
12     HD 158259  Super-Earth>Mini-Neptune>Mini-Neptune>Mini-Nep...
16      HD 20781      Terran>Mini-Neptune>Mini-Neptune>Mini-Neptune
17      HD 20794  Super-Earth>Mini-Neptune>Mini-Neptune>Mini-Nep...
18     HD 215152              Terran>Terran>Super-Earth>Super-Earth
21       HD 3167  Super-Earth>Super-Earth>Mini-Neptune>Mini-Neptune
23      HD 40307  Super-Earth>Mini-Neptune>Mini-Neptune>Mini-Nep...
26        K2-133   Super-Earth>Super-Earth>Mini-Neptune>Super-Earth
28        K2-187        Terran>Super-Earth>Mini-Neptune>Super-Earth
30        K2-268  Super-Earth>Super-Earth>Super-Earth>Mini-Neptu...
39    Kepler-106            Terran>Mini-Neptune>Terran>Mini-Neptune
45   Kepler-1388  Mini-Neptune>Mini-Neptune>Mini-Neptune>Mini-Ne...
48    Kepler-154  Super-Earth>Super-Earth>Mini-Neptune>Mini-Nept...
53    Kepler-172  Mini-Neptune>Mini-Neptune>Mini

### Write pattern data to a JSON file

In [8]:
import json

# Create a list to write to JSON via json.dump(). It will be a list of planet patterns.
data = []
patternNumber = 1

#Create a list of patterns
for p in common_patterns['planet_type_sequence']:
    # Construct inner dictionary
    patternProperties = {
        "pattern" : p.split('>'), # Key is pattern name, value is an ordered list of the planet categories, made by splitting the sequence string
        "hostnames" : df_with_common_patterns[df_with_common_patterns['planet_type_sequence'] == p]['hostname'].tolist(), # Which star systems have this pattern
        "hostname_count": common_patterns[common_patterns['planet_type_sequence'] == p]['num_systems'].tolist()
    }

    data.append({f"pattern_{patternNumber}": patternProperties})
    patternNumber += 1

# Write to a JSON file
filename = "PatternData_MAST.json"
with open(filename, 'w') as f:
   json.dump(data, f, indent=2)



### Write star and planet data to a JSON file

In [9]:


# Create a list to write to JSON via json.dump(). It will be a list of star systems, including planets.
star_system_data = []

#Create a list of star systems
for star in df_with_common_patterns['hostname']:
    # Construct inner dictionaries

    planet_names = final_df[final_df['hostname'] == star]['pl_name'].tolist()
    planets = []
    for planet_name in planet_names:
        planet = {
            "name" : planet_name,
            "radius" : final_df[final_df['pl_name'] == planet_name]['pl_rade'].tolist(), # in Earth Radii
            "orbital_period" : final_df[final_df['pl_name'] == planet_name]['pl_orbper'].tolist(),
            "category": final_df[final_df['pl_name'] == planet_name]['pl_type'].tolist(),
            "order": final_df[final_df['pl_name'] == planet_name]['planet_order'].tolist(),
        }
        planets.append(planet)

    star_system_Properties = {
        #"hostname" : star, # Star name
        "planets": planets
    }

    star_system_data.append({star: star_system_Properties})

# Write to a JSON file
filename = "StarSystemData_MAST.json"
with open(filename, 'w') as f:
   json.dump(star_system_data, f, indent=2)